# Footprint Simulator

In [7]:
from datetime import timedelta
import tkinter

from pathlib import Path
from tkinter import ttk

import floor_plan
import new_functions

class FootprintSimulator:
    N, W, E, S = tkinter.N, tkinter.W, tkinter.E, tkinter.S

    def __init__(self, root, path):
        """
        Parameters
        ----------
        root : tkinter.Tk
        path : pathlib.Path
            A data folder path that includes layout folders.
        
        Returns
        -------
        None
        """
        
        self.root = root
        self.path = path
        self.layout_path = Path()
        self.data_path = Path()
        self.start_flag = False  # whether to be able to start the simulation
        self.simulation_time = timedelta(days = 0)
        
        self.root.title('Footprint Simulator')
        
        # widgets
        self.mainframe = ttk.Frame(root, padding = (3,3,12,12))

        self._time_textvariable = tkinter.StringVar()
        self._time_textvariable.set(str(self.simulation_time))
        self._time_l = ttk.Label(self.mainframe, text = "Time")  # l means label
        self._time = ttk.Label(self.mainframe, textvariable = self._time_textvariable)
        
        self._activity_l = ttk.Label(self.mainframe, text = "Activity")
        self._activity = ttk.Label(self.mainframe, text = "None")
        
        # self._figure = ttk.Frame(self.mainframe, borderwidth = 5, relief = "ridge", width = 200, height = 100)
        self._figure = tkinter.Canvas(self.mainframe, borderwidth = 5, relief = 'ridge', bg = 'white')
        
        self.layouts = []
        for p in self.path.iterdir():
            if p.name != '.ipynb_checkpoints':
                self.layouts.append(p)
        self.layouts_name = [p.name for p in self.layouts]
        self._layouts_name = tkinter.StringVar(value = self.layouts_name)
        self._layout_l = ttk.Label(self.mainframe, text = "Layout")
        self._layout = tkinter.Listbox(self.mainframe, listvariable = self._layouts_name, height = 3)
        self._layout.bind('<<ListboxSelect>>', self.show_data_list)
        
        self.data = []
        self.data_name = []
        self.data_name_listvariable = tkinter.StringVar(value = self.data_name)
        self._data_l = ttk.Label(self.mainframe, text = "Data")
        self._data = tkinter.Listbox(self.mainframe, listvariable = self.data_name_listvariable, height = 3)
        self._data.bind('<<ListboxSelect>>', self.draw_figure)
        
        self._simulation_l = ttk.Label(self.mainframe, text = "Simulation")
        self._simulation = ttk.Button(self.mainframe, text = "Start", command = self.start_simulation)

        # grid geometry manager
        self.mainframe.grid(row = 0, column = 0, sticky = (self.N, self.S, self.E, self.W)) 
        self._time_l.grid(row = 0, column = 0)
        self._time.grid(row = 0, column = 1)
        self._activity_l.grid(row = 0, column = 2)
        self._activity.grid(row = 0, column = 3) 
        self._figure.grid(row = 1, column = 0, columnspan = 6, sticky = (self.N, self.S, self.E, self.W)) 
        self._layout_l.grid(row = 2, column = 0)
        self._layout.grid(row = 2, column = 1)
        self._data_l.grid(row = 2, column = 2)
        self._data.grid(row = 2, column = 3)
        self._simulation_l.grid(row = 2, column = 4)
        self._simulation.grid(row = 2, column = 5) 
        
        # resizable
        self.root.columnconfigure(0, weight = 1)
        self.root.rowconfigure(0, weight = 1)
        self.mainframe.columnconfigure(0, weight = 1)
        self.mainframe.columnconfigure(1, weight = 1)
        self.mainframe.columnconfigure(2, weight = 1)
        self.mainframe.columnconfigure(3, weight = 1)
        self.mainframe.rowconfigure(1, weight = 1)
        
    def show_data_list(self, *args):
        indexes = self._layout.curselection()
        if len(indexes) == 0:
            return
        index = int(indexes[0])
        self.layout_path = self.layouts[index]
        self.data = []
        for p in self.layout_path.iterdir():
            if p.is_dir() and p.name != '.ipynb_checkpoints':
                self.data.append(p)
        self.data_name = [p.name for p in self.data]
        self.data_name_listvariable.set(self.data_name)
        self._figure.delete("all")
        self.start_flag = False
        
    @staticmethod
    def closure_of_point2canvas(reference, scale):
        """
        This is a closure of a function that converts a coordinate in the layout into the coordinate on the canvas.
        
        Parameters
        ----------
        reference : tuple of float
            This is the reference point as the top left coordinate in the layout.
        scale : float
            length in the layout : length on the canvas = 1 : scale.
            
        
        Returns
        -------
        ret : function
            
            This function returns ret = (xx, yy) is the calculated coordinate on the canvas.
        """
        def point2canvas(xy):
            """
            This converts a coordinate in the layout into the coordinate on the canvas.

            Parameters
            ----------
            xy : tuple of float
                xy = (x, y) is the target point in the layout.

            Returns
            -------
            ret : tuple of float
                ret = (xx, yy) is the calculated coordinate on the canvas.
            """
            return (scale*(xy[0] - reference[0]), scale*(reference[1] - xy[1]))
        return point2canvas
        
    def draw_figure(self, *args):
        """
        This draws a layout and sensor arrangements on the canvas.
        """
        indexes = self._data.curselection()
        index = int(indexes[0])
        self.data_path = self.data[index]
        
        # layout
        fp = floor_plan.FloorPlan()
        fp.load_layout(self.layout_path)
        
        scale = 1  # length in the layout : length on the canvas = 1 : scale.
        # reference is the reference point as the top left coordinate in the layout
        reference = (fp.Boundary[0][0] - fp.edge, fp.Boundary[1][1] + fp.edge)
        self.point2canvas = self.closure_of_point2canvas(reference, scale)
        
        # layout
        fp.canvas(self._figure, self.point2canvas)
        
        # sensor
        sensors = new_functions.pickle_load(self.data_path, 'SD_model')
        for s in sensors:
            s.canvas(self._figure, self.point2canvas)
            
        # load activity sequence and walking trajectories
        self.AS = new_functions.pickle_load(self.data_path, 'AS')
        self.WT = new_functions.pickle_load(self.data_path, 'WT')
        self.start_flag = True
        
    def start_simulation(self, *args):
        """
        This starts the footprint simulation.
        """
        if self._simulation['text'] == 'Start':
            if not(self.start_flag):
                return
            self._simulation['text'] = 'Stop'
            
            
        elif self._simulation['text'] == 'Stop':
            self._simulation['text'] = 'Start'
            pass

root = tkinter.Tk()
path = Path('./layout_data')
FootprintSimulator(root, path)
root.mainloop()

In [13]:
fp = floor_plan.FloorPlan()
fp.load_layout(Path('./layout_data/test_layout'))
print(fp.Toil_Bath)

X_lim = [fp.Boundary[0][0] - fp.edge, fp.Boundary[0][1] + fp.edge]
Y_lim = [fp.Boundary[1][0] - fp.edge, fp.Boundary[1][1] + fp.edge]
print(X_lim)
print(Y_lim)

[[420, -330, 90, 120, 'Toilet'], [510, -330, 180, 120, 'Bathroom']]
[-240, 1060]
[-380, 260]


In [8]:
import tkinter as tk
import threading
import time

def long_running_task():
    time.sleep(100)

def button_click():
    # ボタンがクリックされた時の処理
    # 時間のかかる処理を別スレッドで実行する
    thread = threading.Thread(target=long_running_task)
    thread.start()

root = tk.Tk()
button = tk.Button(root, text="Start", command=button_click)
button.pack()
root.mainloop()